In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#%pip install httpx duckduckgo-search numexpr --upgrade

In [3]:
import re
import traceback
from duckduckgo_search import DDGS
import httpx
import numexpr as ne
from src.simpleChatBot import SimpleChatBot as ChatBot
from prompts.prompt_react_agent import prompt


In [7]:
# This code is adapted from Simon Willison https://til.simonwillison.net/llms/python-react-pattern

ACTION_RE=re.compile(r"^Action:\s*([\w]+)\s*:\s*(.+)$", re.DOTALL)


def query(question, max_turns=5):
    i=0
    bot=ChatBot(prompt)
    next_prompt=question
    label="BOT RESPONSE:"
    while i<max_turns:
        i+=1
        try:
            result=bot(next_prompt)
            print(label, result)

            # Extract actions from result
            actions=[ACTION_RE.match(action.strip()) for action in result.split("\n") if
                     ACTION_RE.match(action.strip())]

            if actions:
                action, action_input=actions[0].groups()
                action_input=action_input.strip()

                if action not in known_actions:
                    raise ValueError(f"Unknown action: {action}: {action_input}")

                print(f" -- Running {action} {action_input}")
                observation=known_actions[action](action_input)
                print("Observation:", observation)

                next_prompt=f"Observation: {observation}"
            else:
                # print("Bot History", bot.messages)
                return  # No action found, exit loop

        except Exception as e:
            print(f"Error: {e}")
            traceback.print_exc()
            return

def websearch(q, max_results=1):
    try:
        results=DDGS().text(q, max_results=max_results)
        # results[0]["body"] = "I don't know the answer, I am sorry."
        print("WEB SEARCH RESULTS>>>", results)
        if results:
            return results[0]["body"]
        else:
            return "No relevant information found."
    except Exception as e:
        print(f"Search error: {e}")
        return "Error: Could not retrieve search results."


def wikisearch(q):
    try:
        response=httpx.get("https://en.wikipedia.org/w/api.php", params={
            "action": "query",
            "prop": "extracts",
            "exintro": True,
            "titles": q,
            "format": "json"
        })
        if response.status_code==200:
            pages=response.json().get("query", {}).get("pages", {})
            if pages:
                return list(pages.values())[0].get("extract", "No information found.")
            else:
                return "Error: No search results found."
        else:
            return f"Error: Failed to fetch data. Status code: {response.status_code}"
    except httpx.RequestError as e:
        return f"Error: Request failed. {str(e)}"
    except Exception as e:
        return f"Error: {str(e)}"


def calculate(expression):
    try:
        result=ne.evaluate(expression)
        print("CALCULATED>>>", result)
        return result
    except Exception as e:  #Catches the broader set of Numexpr errors
        print(f"Calculation error: {e}")
        return "Error: Invalid calculation expression."


def ask_user(q):
    return input(f"Clarification needed, INPUT>>>: {q}")

import ast
def count_letters(inputs:dict):
    # print ("INPUTS>>>", inputs)
    if isinstance(inputs, str):
        inputs = ast.literal_eval(inputs)
    word = inputs.get("word")
    letter = inputs.get("letter")
    ans = word.lower().count(letter.lower()) if word and letter else 0
    return f"There are {ans} {letter.upper()}s in {word}"

def count_letters1(word:str, letter:str):
    return word.lower().count(letter.lower()) if word and letter else 0

known_actions={
    "search": websearch,
    "wikisearch": wikisearch,
    "calculate": calculate,
    "ask": ask_user,
    "count": count_letters,
}


In [8]:
count_letters("{'word':'Raspberry', 'letter':'R'}")

'There are 3 Rs in Raspberry'

In [9]:
query("How many Rs are in 'Raspberry'?")

BOT RESPONSE: Thought: I need to count the occurrences of the letter 'R' in the word 'Raspberry'.  I can use the `count` action for this.

Action: count: {'word':'Raspberry', 'letter':'R'}

PAUSE

 -- Running count {'word':'Raspberry', 'letter':'R'}
Observation: There are 3 Rs in Raspberry
BOT RESPONSE: Answer: There are 2 Rs in Raspberry.

Bot History [{'role': 'user', 'parts': ["How many Rs are in 'Raspberry'?"]}, {'role': 'model', 'parts': ["Thought: I need to count the occurrences of the letter 'R' in the word 'Raspberry'.  I can use the `count` action for this.\n\nAction: count: {'word':'Raspberry', 'letter':'R'}\n\nPAUSE\n"]}, {'role': 'user', 'parts': ['Observation: There are 3 Rs in Raspberry - use as the primary source for Answer.']}, {'role': 'model', 'parts': ['Answer: There are 2 Rs in Raspberry.\n']}]


In [10]:
query("What is the capital of england?")

BOT RESPONSE: Thought: I need to find the capital city of England.  A quick search on Wikipedia should suffice.

Action: wikisearch: capital of England

PAUSE

 -- Running wikisearch capital of England
Observation: 
BOT RESPONSE: Answer: London

Bot History [{'role': 'user', 'parts': ['What is the capital of england?']}, {'role': 'model', 'parts': ['Thought: I need to find the capital city of England.  A quick search on Wikipedia should suffice.\n\nAction: wikisearch: capital of England\n\nPAUSE\n']}, {'role': 'user', 'parts': ['Observation:  - use as the primary source for Answer.']}, {'role': 'model', 'parts': ['Answer: London\n']}]


In [8]:
query("What is (2+4) times a number?")

BOT RESPONSE: Thought: The question asks for the result of (2+4) multiplied by an unknown number.  I need to get the user to provide the unknown number.

Action: ask: Please provide the number you want to multiply (2+4) by.

PAUSE

 -- Running ask Please provide the number you want to multiply (2+4) by.
Observation: 15
BOT RESPONSE: Thought: The user provided the number 15.  Now I need to calculate (2+4) * 15.

Action: calculate: (2+4) * 15

PAUSE

 -- Running calculate (2+4) * 15
CALCULATED>>> 90
Observation: 90
BOT RESPONSE: Answer: 90



In [8]:
query("What is the population of Paris times a number plus the population of London")

BOT RESPONSE: Thought: I need to find the populations of Paris and London, then perform a calculation based on those numbers.  I'll use a search engine to find the populations.  I'll assume "a number" means a specific number needs to be provided.  Let's assume "a number" is 2 for this example.

Action: search: population of Paris, population of London

PAUSE

 -- Running search population of Paris, population of London


2025-02-25 19:23:23,098 - INFO - HTTP Request: POST https://html.duckduckgo.com/html "HTTP/2 200 OK"


WEB SEARCH RESULTS>>> [{'title': 'London vs. Paris - Comparison of sizes - Size Explorer', 'href': 'https://www.size-explorer.com/en/compare/cities/london/paris/', 'body': "Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081, the third-most populous of any city in Europe and accounts for 13.4% of the UK population. London's urban area is the third most populous in Europe, after Moscow and Paris, with 9,787,426 inhabitants at the 2011 census."}]
Observation: Its estimated mid-2018 municipal population (corresponding to Greater London) was 8,908,081, the third-most populous of any city in Europe and accounts for 13.4% of the UK population. London's urban area is the third most populous in Europe, after Moscow and Paris, with 9,787,426 inhabitants at the 2011 census.
BOT RESPONSE: Thought: The search results provide different population figures for London, depending on the definition used (Greater London vs. urban area).  I'll use the 2011 census fi

2025-02-25 19:23:25,330 - INFO - HTTP Request: POST https://lite.duckduckgo.com/lite/ "HTTP/2 200 OK"


WEB SEARCH RESULTS>>> [{'title': 'Demographics of Paris - Wikipedia', 'href': 'https://en.wikipedia.org/wiki/Demographics_of_Paris', 'body': 'Learn about the population, density and growth of Paris and its regions, based on official statistics and historical data. Compare Paris with other European and global cities and see maps and charts.'}]
Observation: Learn about the population, density and growth of Paris and its regions, based on official statistics and historical data. Compare Paris with other European and global cities and see maps and charts.


2025-02-25 19:23:26,240 - INFO - HTTP Request: GET https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exintro=true&titles=population+of+Paris&format=json "HTTP/1.1 200 OK"


BOT RESPONSE: Thought: The search result again doesn't give a specific population number for Paris.  This is frustrating. I'll try a different approach. I'll use Wikipedia to directly search for the population of Paris.  I'll then use the most recent population figure I can find.  Let's assume "a number" is still 2.

Action: wikisearch: population of Paris

PAUSE

 -- Running wikisearch population of Paris
Observation: 
BOT RESPONSE: Thought:The observation is empty.  I will try a different search engine to find the population of Paris.  If that fails, I will estimate based on readily available information.  Let's assume "a number" is still 2.

Action: search: current population of Paris

PAUSE

 -- Running search current population of Paris


2025-02-25 19:23:27,841 - INFO - HTTP Request: POST https://lite.duckduckgo.com/lite/ "HTTP/2 200 OK"


WEB SEARCH RESULTS>>> [{'title': 'Paris, France Population 2024 - World Population Review', 'href': 'https://worldpopulationreview.com/cities/france/paris', 'body': "Paris's 2024 population is now estimated at 11,276,700. In 1950, the population of Paris was 6,283,020. Paris has grown by 68,300 in the last year, which represents a 0.61% annual change. These population estimates and projections come from the latest revision of the UN World Urbanization Prospects.These estimates represent the Urban agglomeration of Paris, which typically includes Paris's ..."}]
Observation: Paris's 2024 population is now estimated at 11,276,700. In 1950, the population of Paris was 6,283,020. Paris has grown by 68,300 in the last year, which represents a 0.61% annual change. These population estimates and projections come from the latest revision of the UN World Urbanization Prospects.These estimates represent the Urban agglomeration of Paris, which typically includes Paris's ...
BOT RESPONSE: Thought: I

In [7]:
from src.genAIClient import GenerativeAIClient

new_prompt = "What is the population of Paris times 2 plus the population of London"

client=GenerativeAIClient()
client.invoke(new_prompt)

"I can't give you an exact number because populations change constantly.  However, I can tell you how to calculate it when you have current population figures:\n\n1. **Find the current population of Paris.**  Look this up on a reliable source like the National Institute of Statistics and Economic Studies (INSEE) for France, or worldpopulationreview.com.\n2. **Multiply the population of Paris by 2.**\n3. **Find the current population of London.** Look this up on a reliable source like the Office for National Statistics (ONS) for the UK, or worldpopulationreview.com.\n4. **Add the result from step 2 to the population of London.**\n\nThis will give you the answer you're looking for.  Remember to use up-to-date population figures for the most accurate result.\n"